In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('./data/data.xlsx', index_col=0)
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,2023-04-24 00시,236,251,253,254,265,279,261,279,140,183
1,2023-04-24 01시,211,190,164,192,184,241,208,243,126,138
2,2023-04-24 02시,145,161,115,200,152,184,151,173,83,94
3,2023-04-24 03시,162,170,105,185,119,152,196,141,59,86
4,2023-04-24 04시,200,299,151,47,181,289,362,207,135,169


In [3]:
train_eT = np.array(data.iloc[56,1:].values)
train_eT

array([1005, 1553, 1180, 1682, 1911, 1424, 1628, 1494, 1084, 1454],
      dtype=object)

In [4]:
from BRL import p_matrix, I_matrix, O_matrix
from optimizer4 import solve

In [5]:
from hyperopt import hp, fmin, tpe, Trials

def objective(args):
    w2, w3, w5, w6, w8, w9 = args
    solution = solve(e_T=train_eT, w2=int(w2), w3=int(w3), w5=int(w5), w6=int(w6), w8=int(w8), w9=int(w9))
    if bool(solution): # 해가 존재
        x = np.zeros(66)
        for i in solution:
            x[i] = solution[i]
        e = p_matrix @ x
        diff = np.linalg.norm(train_eT - e)
        return diff
    else: # 해가 존재하지 않음
        return np.Inf

space = [
    hp.quniform('w2', 1, 999, 1),
    hp.quniform('w3', 1, 999, 1),
    hp.quniform('w5', 1, 999, 1),
    hp.quniform('w6', 1, 999, 1),
    hp.quniform('w8', 1, 999, 1),
    hp.quniform('w9', 1, 999, 1),
]

results = []
for i in range(100):
    try:
        trials = Trials()
        result = fmin(
            fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50
        )
        
        results.append(result)
    except Exception as e:
        print(f"Iteration {i}: 에러가 발생했습니다. 다음 반복으로 이동합니다. 오류 내용: {e}")

100%|██████████| 50/50 [00:04<00:00, 11.67trial/s, best loss: inf]


In [6]:
len(results)

100

In [7]:
errors = []
for result in results:
    solution = solve(e_T=train_eT, w2=int(result['w2']), w3=int(result['w3']), w5=int(result['w5']),\
        w6=int(result['w6']), w8=int(result['w8']), w9=int(result['w9']))
    x = np.zeros(66)
    for i in solution:
        x[i] = solution[i]
    e = p_matrix @ x
    errors.append(np.linalg.norm(train_eT - e))

In [8]:
errors

[4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,
 4636.231982979281,


In [9]:
zeros = [i for i, error in enumerate(errors) if error == 0.0]
optimal = []
for zero in zeros:
    optimal.append(results[zero])

In [10]:
import pickle
with open('zeros.pickle', 'wb') as f:
    pickle.dump(optimal, f)

In [11]:
with open('zeros.pickle', 'rb') as f:
    zz = pickle.load(f)
print(zz)

[]


In [12]:
solution = solve(e_T=train_eT, w2=626, w3=11, w5=1, w6=12, w8=288, w9=78)
x = np.zeros(66)
for i in solution:
    x[i] = solution[i]
e = p_matrix@x
print(e)
print(train_eT)
print(np.linalg.norm(train_eT-e))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1005 1553 1180 1682 1911 1424 1628 1494 1084 1454]
4636.231982979281
